## NumPy

### Généralités

Nous avons brièvement importé le paquet Numpy dans le [tutoriel 4](./Tutorial_4_Imports_functions.ipynb) afin de permettre de diviser une liste d'entiers par un nombre de manière efficace et pratique, alors que ces opérations ne fonctionnent pas sur des objets `list`. Mais NumPy est un très gros module et Python et contient un très grand nombre de fonctions permettant de travailler avec les nombres, entre autres : 

* Des méthodes permettant de créer et manipuler des vecteurs et des matrices, généralisés sous la forme de `np.ndarray()`. Les amateurs de MATLAB seront heureux de les retrouver :-)
* Des définitions pour quasiment toutes les fonctions mathématiques usuelles, y compris les fonctions hyperboliques et leurs réciproques (par exemple `arctanh`)
* Quelques fonctions d'intérêt pour manipuler les valeurs de vos tableaux, par exemple pour sommer les élements, trouver un élément particulier ou supprimer les doublons.
* Quelques fonctions d'intérêt pour les statistiques et les nombres aléatoires avec le sous-paquet `numpy.random`.
* Et les fonctions et la syntaxe associée aux _transformées de Fourier_, `numpy.fft`

Sachez qu'une grande partie du code de NumPy est en fait écrit en C, avec une sur-couche de Python par-dessus. Les fonctions NumPy sont donc bien optimisées pour le travail avec les grandes matrices, et peuvent être encore _accélérées_ si le besoin s'en ressent en utilisant des modules supplémentaires, par exemple [Numba](https://numba.pydata.org/) qui va compiler votre code et éventuellement le _paralléliser_ sur les coeurs de votre processeur, ou même votre carte graphique.

-------------------------------

### Les objets `np.ndarray`

Nous l'avons brièvement aperçue précédemment. Quand les éléments des `np.ndarray` sont des nombres, les `np.ndarray` représentent des vecteurs, des matrices ou même des tenseurs d'ordre (de dimension) plus élevée. Ils fonctionnent presque comme les listes, et toute la syntaxe associée va donc leur ressembler fortement, __hormis leur comportement avec les opérateurs usuels (+, *, -, / et **) et leur indexation__. On crée de tels objets assez simplement par exemple à partir d'une liste et de la fonction `np.array()`: 

In [ ]:
import numpy as np
x = np.array([3,4,5])

print(x)
print(type(x))

#### Indexation et _coupes_ /!\\


##### Indexation et coupes simples (à un indice)

Les `np.ndarray` s'indexent d'une manière différente que les objets `list` en Python. Pour les tableaux à une dimension, toutefois, rien de bien méchant, on peut facilement récupérer un élément, ou une _coupe_ ([cf Tutoriel 2](./Tutorial_2_ListsTuplesDicts.ipynb#Coupes)) et les résultats sont identiques à ceux obtenus pour les listes :

In [ ]:
my_list = [1,2,6,9,11,-3]
my_array = np.array(my_list)
print(my_list[5])
print(my_array[5])
print(my_list[2:4])
print(my_array[2:4])

Construisons une liste de 3 listes contenant chacune 3 nombres appelée `mylist_of_lists` (cf ci-dessous). Ma liste de listes peut être vue comme une matrice $3 \times 3$. Regardons ce qui arrive lorsque j'essaie d'indicer ma liste de liste et le tableau de manière 'simple', c'est à dire juste avec un indice : 

In [ ]:
mylist_of_lists = [[1,7,2],[3,4,5],[6,7,8]]
my_matrix = np.array(mylist_of_lists)

print(mylist_of_lists[1])
print(my_matrix[1])

Les résultats sont identiques, ce qui est pour l'instant plutôt rassurant ! Il est d'ailleurs possible de faire la même chose avec une coupe, par exemple en remplaçant le `[1]` par `[:]`, et les résultats seront (aux détails de l'affichage près) identiques. 



##### Coupes multi-dimensionnelles des tableaux à d > 1  /!\\

Supposons maintenant que Je voudrais faire une _coupe multidimensionnelle_ de cette matrice : dans mon cas précis, je souhaiterais ne garder que le bloc $2\times 2$ correspondant au nombres 'en bas à droite' de la matrice initiale. On peut assez facilement sélectionner les deux dernières lignes de la liste de liste avec la _coupe_ `[1:3]`, mais il n'est pas possible de directement sélectionner certaines 'colonnes de la matrice' dans la liste de listes. Voyez plutôt : 

In [ ]:
mylist_of_lists = [[1,7,2],[3,9,4],[11,-7,0]]

print(mylist_of_lists[1:3])                                # Almost, but not quite
print(mylist_of_lists[1:3][1:3])                           # Does not do what we want ...

Pour faire une sélection sur les colonnes, il nous faudra examiner un par les deux derniers élements de la liste de listes (c'est à dire les deux dernières lignes de la matrice $3\times 3$) et pour chacune de ces deux lignes (ou _sous-listes_), ne garder que les deux derniers nombres. On ne peut effectuer une telle opération qu'avec une boucle `for` :

In [ ]:
mylist_of_lists = [[1,7,2],[3,9,4],[11,-7,0]]

sub_list_of_lists = []
for row in mylist_of_lists[1:]:
    sub_list_of_lists.append(row[1:])
print(sub_list_of_lists)                                           # Too Complicated ! 

_NumPy_ va ici grandement nous aider, car il permet de faire des coupes selon plusieurs dimensions en utilisant la syntaxe 
```
sub_matrix = my_matrix[a:b,c:d:e,f:g, ...]
```
Ce qui va sélectionner 
* les _lignes_ $a$ à $b-1$ dans la première dimension
* les _colonnes_ $c, c+e, c+2e, \ldots, d-1$ dans la deuxième dimension
* les _couches_ $f$ à $g-1$ dans la troisième dimension
* etc. etc. pour les dimensions suivantes 

Pour le cas de l'exemple précédent : 

In [ ]:
mylist_of_lists = [[1,7,2],[3,9,4],[11,-7,0]]
my_matrix = np.array(mylist_of_lists)
print(my_matrix[1:3,1:3])   # Quick and easy ! 

##### Indicage logique

Il existe des [méthodes plus avancées pour indicer vos tableaux](https://numpy.org/doc/stable/user/basics.indexing.html#advanced-indexing). Je ne vais pas toutes les détailler ici, car ces méthodes vont rendre ce cours vraiment confus, et je pense qu'il y a déjà beaucoup d'informations dans ce long module. N'hésitez pas toutefois à aller lire la page de documentation ci-dessus si vous avez besoin d'effectuer un indiçage bien particulier qui n'est pas décrit ici :-) . 

Je vais me borner à expliquer l'indiçage _logique_ (ou booléen), qui est possible sous NumPy comme sous MATLAB. Le principe est le suivant : plutôt d'indicer ou de découper un tableau avec des _indices_ `[i,j,k]` ou des coupes `[a:b:c, d:e, ...]`, on va 'indicer' un tableau `x` avec un autre tableau `cond` (pour _condition_) de la même taille que `x` et qui va contenir des [_booléens_](./Tutorial_1_SimpleThings.ipynb#La-vérité-est-ailleurs-:-Booléens-et-expressions). En évaluant l'expression `x[cond]`, on va filtrer les éléments de `x` et ne garder que ceux qui correspondent à `cond == True`. Si cela vous fait mal à la tête, essayez plutôt le schéma et l'exemple suivants :-)

![img](./resources/Boolean%20Indexing.png)

In [ ]:
x    = np.array([7   , 12   ,0    ,66    ,23    , 1   , 9   , 46   , -7   , -89])
cond = np.array([True, False, True, False, False, True, True, False, False, False])

print(x[cond])

Vous pouvez donc voir que le tableau `cond` agit en fait comme un _filtre de sélection_ des valeurs de x.

__Exercices__ : 

* En utilisant une coupe multidimensionnelle, extrayez les quatres valeurs des 'coins' de la matrice `my_matrix` ci-dessus.
* Reprenez le vecteur `x` ci-dessus, et utilisez l'indiçage logique pour créer un tableau `xpos` ne contenant que des valeurs positives.  

#### Sommes et multiplications /!\\

__Contrairement__ aux listes à nouveau, la multiplication et l'addition de scalaires à un `np.ndarray` fonctionnent également comme sous MATLAB, et ce quel que soit la 'dimension' du tableau: 
* Pour `+` et `-`, on ajoute (soustrait) donc à chacun des éléments du tableau le scalaire en question
* Pour `*` et `/`, on multiplie (divise) donc chacun des éléments du tableau par le scalaire en question
* Pour `**`, on met chacun des éléments du tableau à la puissance en question (__le `.^` de MATLAB__ !)

In [ ]:
vector = np.array([3,6,1])
matrix = np.array([[1,2,3],[4,5,6],[7,8,9]])

print(vector + 3.15)
print(-5.7*vector)
print(-2.1*matrix + 6.32)
print(matrix**2)

Quand les tableaux ont des dimensions compatibles, on peut les multiplier entre eux, et par défaut cette multiplication se fait _élément par élément_ (le `.*` préféré des amateurs de MATLAB) : 


In [ ]:
vector_1 = np.array([3,6,2])
vector_2 = np.array([6,4,1])
matrix = np.array([[1,1,1],[1,1,1],[1,1,1],[2,2,2]])

print(vector_1*vector_2)
print(vector_1*matrix)

* Dans le premier cas, le comportement est assez logique, on multiplie les éléments de chaque vecteur 'un par un'.
* Dans le deuxième cas, NumPy a automatiquement cherché une dimension de `matrix` dont la longueur (3) est compatible avec celle (3) de `vector_1`, et ici, c'est la dimension n°2 (les colonnes). NumPy va ensuite _étendre_ `vector_1` dans l'autre dimension (ou les autres dimensions) pour obtenir la même forme que `matrix`, et enfin multiplier un par un les éléments de `matrix` et de `vector_1` étendu. 

#### Opérations sur les dimensions

##### Listage

On peut demander à NumPy de renvoyer les propriétés de forme des objets `np.ndarray` en utilisant les fonctions : 
* `np.shape()` : va renvoyer la longueur d'un tableau dans chacune des dimensions de celui-ci
* `np.size()` : va renvoyer le nombre d'éléments total du tableau
* `np.ndim()` : va renvoyer le nombre de dimensions du tableau 

In [ ]:
tensor = np.array([[[1,2],[2,3]],[[12,14],[15,19]],[[-2,-1],[-3,-6]]])
print(np.shape(tensor))
print(np.size(tensor))
print(np.ndim(tensor))

__Mini-quiz__: 
* Que renvoie la commande `len(tensor)` ? Comment interprétez-vous le résultat ?
* Essayez de re-créer les fonctions `np.size()` et `np.ndim()` à partir de `np.shape()` en utilisant des fonctions Python et des fonctions Numpy, par ex. `np.sum()`

##### Rajouter des dimensions

On peut assez facilement rajouter des dimensions en exécutant la fonction `np.atleast_nd()` (avec $n = 1,2,3$). En fonction de la dimension de votre tableau initial, les dimensions _en plus_ ne seront pas forcément rajoutées là où vous vous y attendez ! Voyez par exemple, pour un tableau à une dimension (un vecteur) transformé en tableau à trois dimensions :

In [ ]:
x = np.array([1,2,6,8,2])
y = np.atleast_2d(x)
z = np.atleast_3d(x)
print(np.shape(x))
print(np.shape(y))
print(np.shape(z))

Vous pouvez également rajouter _à la main_ des dimensions en plus dans votre tableau en l'indexant avec une coupe qui renverrait normalement la matrice entière `[:,...,:]` et en rajoutant un `np.newaxis` à l'endroit de la dimension que vous souhaitez rajouter. Par exemple, on peut ici rajouter une troisième dimension à une matrice 'en premier', ou même 'au milieu' de la matrice initiale !

In [ ]:
matrix = np.array([[1,2,3],[6,5,4]])
bigger_matrix = matrix[:,np.newaxis,:]
other_bigger_matrix = matrix[np.newaxis,:,:]
print(np.shape(bigger_matrix))
print(np.shape(other_bigger_matrix))

##### Permuter des dimensions

Si vous avez le malheur de travailler avec des objets à plus de deux dimensions (j'en suis navré par avance), sachez que tout n'est pas perdu et vous pouvez échanger et déplacer les _dimensions_ de vos objets de manière efficace :
* `np.moveaxis()` qui déplace un ou plusieurs axes vers une ou plusieurs positions données,
* `np.swapaxes()` qui échange deux listes d'axes
* `np.transpose()` qui, enfin, inverse les axes (les axes 0,1,2,...,n deviennent les axes n,n-1,...,0). __La fonction `np.transpose()` fonctionne comme la transposée _normale_ pour des tableaux de dimension 2__.

Le résultat fait souvent mal à la tête :-), voyez donc :  

In [ ]:
tensor = np.array([[[1,2,3],[4,5,6]],[[7,8,9],[10,11,12]],[[13,14,15],[16,17,18]],[[19,20,21],[22,23,24]]])

print('Initial, shape = ' + str(np.shape(tensor)))
print(tensor)

moved = np.moveaxis(tensor, 0, -1) # Moving the first axis (index 0, rows) to third axis (index -1, layers)
print('-------')
print('Moved, shape = ' + str(np.shape(moved)))
print(moved)

swapped = np.swapaxes(tensor, 0, -1) # Exchanging the first (index 0, rows) and the last axis (index -1, layers)
print('-------')
print('Swapped, shape = ' + str(np.shape(swapped)))
print(swapped)

transposed = np.transpose(tensor) # NOTE : tensor.T also works
print('-------')
print('Transposed, shape = ' + str(np.shape(transposed)))
print(transposed)

_Note_ : Pour les habitués de MATLAB, la fonction `np.transpose()` ne fonctionne pas avec les vecteurs. En effet, sous MATLAB, tous les objets ont au moins 2 dimensions par défaut, mais pas sous NumPy, où les objets à 0 et 1 dimensions existent vraiment, et ... il est difficile de permuter les dimensions d'objets n'ayant qu'une seule dimension ! Pour effectuer une réelle transposition d'un vecteur, il faudra utiliser _d'abord_ la fonction `np.atleast_2d()` sur votre vecteur pour que la transposition se fasse correctement.

In [ ]:
vector = np.array([1,2,6,2,1])

print('Original, shape = ' + str(np.shape(vector)))
print(vector)

print('Transposed --- Naive, shape : ' + str(np.shape(np.transpose(vector))))
print(np.transpose(vector))

print('Transposed --- OK, shape : ' + str(np.shape(np.transpose(np.atleast_2d(vector)))))
print(np.transpose(np.atleast_2d(vector)))

##### Redimensionner des `np.ndarray`

Vous pouvez enfin redimensionner les tableaux en question. Par exemple, vous pouvez transformer une matrice $n \times m$ en une autre matrice de taille $p \times q$, à condition bien entendu que $n\times m = p\times q$. Pour cela, on utilisera la fonction `np.reshape()`. Celle-ci prend en première entrée le tableau à redimensionner, et en deuxième les entiers correspondant à la nouvelle forme attendue pour la matrice. 

_Quelques remarques_

* Les nouvelles dimensions doivent être transmises sous forme de liste ou de tuple, c'est pourquoi on doit mettre ces dimensions entre parenthèses `(`,`)` ou crochets `[`,`]`.
* Il est possible de rajouter une dimension au passage, si $n \times m = p \times q \times r$ ! En supprimer une est également possible.
* Si l'ordre dans lequel les éléments de la matrice sont réarrangés ne vous convient pas, vous pouvez toujours essayer de _transposer_ ou de _bouger des dimensions_ de la matrice avant de la redimensionner !
* Il est possible d'omettre une dimension et laisser NumPy calculer combien d'éléments doivent être _placés_ dans cette dimension. Dans ce cas, mettez un $-1$ au lieu de $p$ ou $q$.

In [ ]:
my_matrix = np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])

print('---- Original')
print(my_matrix)
print('---- Reshaped')
print(np.reshape(my_matrix,[2,6]))  
print('---- Transposed Reshaped')        
print(np.reshape(np.transpose(my_matrix),[2,6]))
print('---- 3d Reshape')
print(np.reshape(my_matrix, [2,2,3]))

Vous pouvez enfin aplatir un tableau complètement de deux manières différentes. Pour cela, vous pourrez utiliser les _méthodes_ `.flatten()` ou `.ravel()` disponibles pour les tableaux `np.ndarray`. La différence entre ces deux méthodes est que l'objet créé par `.ravel()` va rester lié à la matrice initiale, et va par exemple permettre de la mettre à jour, tandis que celui créé par `.flatten()` sera complètement distinct de la matrice initiale. C'est du chinois ? Allez voir le [Tutoriel 2](./Tutorial_2_ListsTuplesDicts.ipynb#Python-et-la-mutabilité) !

In [ ]:
my_matrix = np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])

flattened = my_matrix.flatten()
unraveled = my_matrix.ravel()

print('Flattened :')
print(flattened)

print('Unraveled : ')
print(unraveled)

print('-Modifying flattened-')
flattened[3] = -5
print(my_matrix) # my_matrix not impacted by changes of flattened

print('-Modifying Unraveled-')
unraveled[3] = -7
print(my_matrix) # my_matrix not impacted by changes of flattened

__Exercice__ : 

* En utilisant les fonctions `np.reshape()`, `np.flatten()` et  `np.transpose()`, essayez de transformer le vecteur $v$ ci-dessous en un autre vecteur dont les éléments impairs sont au début et les éléments pairs à la fin.
* Horreur ! Ma matrice $m$ a tous ses éléments dans le désordre. Essayez de déplacer ou d'échanger des axes avec `np.transpose()`, `np.swapaxes()`, `np.moveaxis()` puis aplatissez la matrice en question afin d'avoir des élements bien ordonnés.

In [ ]:
v = np.array([3,4,5,6,7,8,9,10,11,12])
m = np.array([[[1,10,19],[2,11,20],[3,12,21]],[[4,13,22],[5,14,23],[6,15,24]], [[7,16,25],[8,17,24],[9,18,27]]])

#### Remplir des `np.ndarray` efficacement

##### Vecteurs

Remplir les tableaux avec la commande `np.array()` est particulièrement fastidieux. Il serait par exemple intéressant de pouvoir emplir un vecteur avec des valeurs régulièrement espacées entre deux valeurs $a$ et $b$. C'est le cas, par exemple, si vous voulez avoir une idée de ce qu'une fonction $f$ vaut sur un intervalle $[a,b]$. Nous traçons de tels graphes dans [l'Application B](./Application_B_Plotting.ipynb).

##### Ici -> MATPLOTLIB

Obtenir des valeurs espacées régulièrement entre $a$ et $b$ est possible, et ce de plusieurs manières, grâce aux fonctions : 

* `np.arange(a, b, r)`, qui crée un tableau de valeurs correspondant à $a, a+r, a+2r, \ldots ...$ jusqu'à atteindre $b$ (exclus)
* `np.linspace(a, b, N)`, qui crée un tableau 1d contenant $N$ valeurs régulièrement espacées linéairement entre $a$ et $b$ (__les deux inclus__)
* `np.logspace(a, b, N)`, qui crée un tableau 1d contenant $N$ valeurs régulièrement et __logarithmiquement__ espacées entre $\log_{10}(a)$ et $\log_{10}(b)$. 

Ces fonctions sont particulièrement pratiques pour créer des valeurs d'abscisse (l'axe $x$ des graphes) sur lesquels vous allez ensuite appliquer des fonctions mathématiques pour obtenir des valeurs d'ordonnée (l'axe $y$ des graphes).

Voyez plutôt : 

In [ ]:
print(np.arange(1.2,23,3.5))
print(np.linspace(0,24,13))
print(np.logspace(-5,0,11))

Vous aurez probablement besoin de quelques essais avant de tomber sur ce qu'il vous faut exactement, mais je vous fais confiance :-).

##### Matrices : initialisation

De nombreuses fonctions permettent de créer et de remplir des matrices de tailles arbitraires. 

* La plus simple d'entre elles, `np.zeros()`, va créer une matrice remplie de zéros de la taille qui nous intéresse. Cette taille sera spécifiée sous la forme d'une liste ou de tuple. 
* De la même manière, `np.ones()` va créer une matrice remplie de 1, en lui spécifiant en entrée une liste de dimensions.
* La fonction `np.eye()` va créer une matrice identité de taille donnée. Cette fois-ci, comme c'est une matrice identité, on ne lui donne en entrée qu'un entier.

In [ ]:
requested_size = (3,2)
print(np.zeros(requested_size))
print(np.ones([4,2]))
print(np.eye(5))

##### Matrices : pavage et empilement 

Vous pouvez également concaténer, empiler des matrices ou bien répéter leurs lignes et colonnes dans les dimensions qui vous intéressent. Il est notamment possible de (d') :

1. Empiler des matrices verticalement (empiler leurs _lignes_) grâce à la commande `np.vstack`, qui prend en entrée une séquence (`tuple` ou `list`) de matrices qui ont donc toutes le même nombre de colonnes.
2. Faire la même chose horizontalement avec `np.hstack`. Les matrices de la séquence en entrée doivent alors toutes avoir le même nombre de lignes !
3. _Paver_ une matrice ou un vecteur dans une ou plusieurs directions avec la commande `np.tile`. La commande prend en entrée une matrice, puis une liste d'entiers $[a,b,c,\ldots]$, qui va spécifier qu'on répète la matrice entière $a$ fois au niveau des lignes, $b$ fois au niveau des colonnes, $c$ fois au niveau des couches, etc.
4. _Répéter_ votre tableau _ligne par ligne_ avec la commande `np.repeat`. Cette fonction va répéter $a$ fois des lignes, des colonnes ou des couches d'une matrice $M$. On doit donc lui spécifier une matrice en entrée, puis un nombre de répétitions, et enfin un axe le long duquel répéter les valeurs avec la clé `axis=...`. __Si cet axe n'est pas précisé, la matrice est _aplatie_ avant que `np.repeat() ne s'effectue !__

In [ ]:
print('Vstack ---')
first_matrix = np.array([[1,2,3],[4,5,6]])    # Size 2 (rows) x3 (columns)
second_matrix = np.eye(3)                     # Size 3 (rows) x3 (columns)
print(np.vstack((first_matrix,second_matrix)))

print('Hstack ---')
third_matrix = np.ones((2,2))                 # Size 2 (rows) x 2 (columns)
fourth_matrix = np.zeros((2,1))               # Size 2 (rows) x 1 (columns)
print(np.hstack((first_matrix,third_matrix,fourth_matrix)))

print('Tile ---')
print(np.tile(first_matrix, [1, 3]))            # Repeat matrix 1x along lines and 3x along columns

print('Repeat ---')
print(np.repeat(first_matrix, 3, axis=0))       # Repeat each row 3x

__Exercices__ : 

* À partir des deux tableaux 1d de coordonnées $x$ et $y$ ci_dessous, créez un tableau à deux lignes $XY$ contenant tous les couples de coordonnées $(x,y)$ possibles.
* Créez une [matrice de _Vandermonde_](https://fr.wikipedia.org/wiki/Matrice_de_Vandermonde) carrée à partir d'une boucle `for` et du vecteur `vdm` spécifié ci-dessous. 

In [ ]:
x = np.arange(0,7)
y = np.linspace(-5,0,10)

vdm = np.array([2,-1,3,-2,-3,0])                              # This is not 'vie de m----' :-)

------------------------------------------

### Constantes et fonctions mathématiques


#### Constantes mathématiques 

NumPy contient également des constantes utiles en mathématiques et non présentes en Python, notamment $\pi, e, \gamma, \pm \infty$ (appelé `inf` par NumPy) et `NaN` (pour _not a number_ en anglais). Les premiers objets seront assez familiers aux mathématiciens, et NumPy effectue correctement la plupart des opérations arithmétiques dessus (cf. ci-dessous). Le `np.NaN` va servir à remplacer les formes indéterminées (du type $0/0$ ou $\infty - \infty$, ou des valeurs de tableaux dont ont sait _a priori_ qu'elles sont incorrectes ou qu'elles ont des problèmes. Les `NaN` ont une tendance à se `propager` dans les tableaux, car toutes les opérations les impliquant renvoient `NaN`.

_Notes_: 

* Pour NumPy, tous les infinis sont les mêmes donc `np.inf == np.inf` va renvoyer `True`
* Par contre, les `Nan` ne sont pas tous identiques, donc `np.nan == np.nan` va renvoyer `False`

In [ ]:
import numpy as np

print('pi = ' + str(np.pi))
print('Inf - any number = ' + str(np.inf - 100000)) # Choose any number here
print('Inf/Inf = ' + str(np.inf/np.inf))            # Indeterminate form --> NaN
print('Nan+Inf = ' + str(np.nan+np.inf))            # Operations involving NaN almost always result in NaN
print('Inf == Inf test : ' + str(np.inf == np.inf)) # Comparisons between Inf and itself
print('Nan == Nan test : ' + str(np.nan == np.nan)) # Comparisons between NaN and itself

Deux fonctions NumPy sont spécifiquement pensées pour travailler avec $\infty$ et `NaN`:  
* `np.isfinite()` va renvoyer, pour chaque élément d'un tableau, `False` si cet élément est $\pm\infty$ ou `NaN`
* `np.isnan()` renvoie, de la même manière, `True` seulement pour les élements du tableaux qui valent `NaN` 

Avec l'exemple ci-dessous, vous pourrez également constater que quand vous sommez, multipliez, etc. des tableaux contenant des valeurs infinies ou `NaN`, NumPy vous renvoie généralement un avertissement (_warning_) et que vous feriez bien de jeter un oeil au résultat.

In [ ]:
u = np.array([7 , np.inf, -np.inf, -np.inf, np.nan])
v = np.array([23, 28    , -np.inf, +np.inf, +np.inf])

print('Finiteness : ' + str(np.isfinite(u + v)))
print('Nan-ness   : ' + str(np.isnan(u + v)))

__Exercices__: 

* Que valent $0/0$ ;  $0 \times$ `np.nan` ;  $0 /$ `np.nan` sous NumPy ?
* Essayez de créer une fonction `nanmean` qui va moyenner toutes les valeurs de `dirty_table`, mais en ignorant les `np.nan`* . Vous pouvez par exemple vous aider de l'[indiçage booléen](#Indicage-logique). 

<small>* Cette fonction existe en NumPy, [`np.nanmean`](https://numpy.org/doc/stable/reference/generated/numpy.nanmean.html) !</small>

In [ ]:
dirty_table = np.array([np.nan, 7, 3, 0, -5, -9, -183, np.nan, 28, 256, np.nan])

#### Fonctions mathématiques usuelles

Nous les avons déjà entre-aperçues dans le [Tutorial 4](./Tutorial_4_Imports_functions.ipynb), et elles sont bien là : 

* Les fonctions exponentielles et logarithme avec `np.exp()` et `np.log()`, également en base 10 : `np.log10()`
* Les fonctions trigonométriques `np.sin()`, `np.cos()`, `np.tan()`, et leurs réciproques `np.arccos()`, `np.arcsin()`, `np.arctan()`. Il existe notamment une fonction `np.arctan2(y,x)`, qui est l'équivalent de l'_argument_ des nombres complexes $z = x + iy$ et permet de retrouver l'angle d'un point $M=(x,y)$ dans un plan à deux dimensions. 
* Les fonctions hyperboliques `np.sinh()`, `np.cosh()` et `np.tanh()` et également leurs réciproques `np.arccosh()`, ...
* La fonction racine carrée, `np.sqrt()`, même si l'utilisation de la puissance 0.5 `**(1/2)` était déjà possible.

##### ICI --> Scipy
 
Si votre fonction préférée n'est pas disponible dans NumPy (elle doit être dans ce cas assez spécifique ou vraiment alambiquée), ne perdez pas espoir, le paquet [_SciPy_](./Application_C_Scipy.ipynb) en contient encore plus.

In [ ]:
print(np.sin(np.pi/6))
print(np.log(2*3) == np.log(2) + np.log(3))
print(np.tanh(np.inf))  # Behaves nicely at x -> +inf

__Exercices__ : 

* Calculez les valeurs de ${\rm sinc}(t) = \sin(t)/t$ pour 50 valeurs de $t$ allant de $0$ (inclus) à $\pi$ (inclus). À quelle valeur vous attendez-vous en $t=0$ ? Cela correspond-il à la limite mathématique de la fonction ? Que se passe-t-il pour $t\to \infty$ (c'est à dire `x = np.inf` en NumPy) ? Essayez d'interpréter le résultat.
* (difficile !) À partir des coordonnées $(x,y)$ ci-dessous de points $M_i$ (par exemple le centre de particules), déterminez les distances et les angles entre couples de points $(M_i, M_{j\neq i})$. _Indice_ : commencez par construire des matrices $[d_x]_{ij} = x_j - x_i$ et $[d_y]_{ij} = y_j - y_i$ en utilisant les fonctions abordées dans [une des sections précédentes](#Remplir-des-np.ndarray-efficacement).

In [ ]:
# Cardinal function sinc

# Coordinates to be investigated:
x = np.array([+2,+5,+3,+4,+6,+3,+4,+5])
y = np.array([+5,+6,+4,+7,+5,+6,+5,+4])

In [ ]:
ii